In [1]:
import meshzoo
import torch

from torchfem import Shell
from torchfem.materials import IsotropicElasticityPlaneStress

# Cantilever plate 

A plate of dimensions 100mm x 50mm x 0.5mm clamped at one end. It is subjected to concentrated forces of 5N at each corner at the tip. 

A reference solution is computed with ABAQUS for the following shell elements: 

| Type   | u_max | S_max |
|--------|-------|-------|
| S3     | 30.36 | 423.6 |
| STRI3  | 30.64 | 461.7 |
| STRI65 | 30.90 | 481.4 |

In [2]:
# Material parameters
material = IsotropicElasticityPlaneStress(E=200000.0, nu=0.3)

# Define nodes and element of the plate
X = torch.linspace(0.0, 100.0, 10)
Y = torch.linspace(0.0, 50.0, 5)
points, cells = meshzoo.rectangle_tri(X, Y, variant="zigzag")
N = len(points)
nodes = torch.hstack([torch.tensor(points, dtype=torch.float32), torch.zeros((N, 1))])
elements = torch.tensor(cells.tolist())

# Define Shell model
cantilever = Shell(nodes, elements, material)

# Define masks for boundaries
left = nodes[:, 0] < 0.01
corner_1 = (nodes[:, 0] > 99.9) & (nodes[:, 1] < 0.01)
corner_2 = (nodes[:, 0] > 99.9) & (nodes[:, 1] > 49.99)

# Apply load boundary conditions
cantilever.forces[corner_1, 2] = 5
cantilever.forces[corner_2, 2] = 5

# Apply displacement boundary conditions
cantilever.constraints[left] = True

# Thickness
cantilever.thickness[:] = 0.5


# Solve and visualize results
u, f = cantilever.solve()
mises = cantilever.compute_stress(u, z=0.25, mises=True)
print(torch.max(u[:, 0:3]))
print(torch.max(mises))
cantilever.plot(u[:, 0:3], node_property={"u": torch.linalg.norm(u[:, 0:3], dim=1)})
cantilever.plot(u[:, 0:3], element_property={"mises": mises}, thickness=True)

tensor(30.7007)
tensor(430.3632)


Widget(value='<iframe src="http://localhost:55617/index.html?ui=P_0x164fb6860_0&reconnect=auto" class="pyvista…

Widget(value='<iframe src="http://localhost:55617/index.html?ui=P_0x167abe5f0_1&reconnect=auto" class="pyvista…

# Center-loaded plate

A square plate of 100x100 is fully clamped at all edges and subjected to a concentrated force F=16.367 at the center. A symmetric quarter model is computed. 


A reference solution is computed with ABAQUS for the following shell elements: 

| Type   | u_max  | 
|--------|--------| 
| S3     | -3.773 |
| STRI3  | -4.041 | 
| STRI65 | -4.486 | 
 

In [3]:
# Constants
L = 100.0
F = -16.367

# Material parameters (Cs is transverse shear stiffness)
mat = IsotropicElasticityPlaneStress(E=10000.0, nu=0.3)

# Define plate
X = torch.linspace(0.0, 0.5 * L, 9)
Y = torch.linspace(0.0, 0.5 * L, 9)
points, cells = meshzoo.rectangle_tri(X, Y, variant="zigzag")
N = len(points)
nodes = torch.hstack([torch.tensor(points, dtype=torch.float32), torch.zeros((N, 1))])
elements = torch.tensor(cells.tolist())

# Create Shell model
plate = Shell(nodes, elements, mat)

# Boundaries
top = nodes[:, 1] > (L / 2.0 - 0.01)
bottom = nodes[:, 1] < 0.01
left = nodes[:, 0] < 0.01
right = nodes[:, 0] > (L / 2.0 - 0.01)

# Boundary conditions
plate.forces[0, 2] = F
plate.constraints[top] = True
plate.constraints[right] = True
plate.constraints[left, 0] = True
plate.constraints[left, 4] = True
plate.constraints[left, 5] = True
plate.constraints[bottom, 1] = True
plate.constraints[bottom, 3] = True
plate.constraints[bottom, 5] = True


# Solve
u, f = plate.solve()
print(u.min())
mises = plate.compute_stress(u, z=0.25, mises=True)
plate.plot(
    u[:, 0:3],
    node_property={"u": torch.linalg.norm(u[:, 0:3], dim=1)},
    mirror=[True, True, False],
)
plate.plot(
    u[:, 0:3],
    element_property={"mises": mises},
    thickness=True,
    mirror=[True, True, False],
)

tensor(-3.9851)


Widget(value='<iframe src="http://localhost:55617/index.html?ui=P_0x1500fdfc0_2&reconnect=auto" class="pyvista…

Widget(value='<iframe src="http://localhost:55617/index.html?ui=P_0x1500ffac0_3&reconnect=auto" class="pyvista…